# Grafted Particle Composite

description

### Concepts Used
- blergh

### Tools Used
- blergh

References
------------
1. Modica, K.J.; Martin, T.B.; Jayaraman, A.J.; Effect of Architecture on the Structure
   and Interactions of Polymer Grafted Particles: Theory and Simulation, Macromolecules, 
   2017, 50 (12), pp 4854-4866


Global Imports and Setup
---------------------------

In [ ]:
import holoviews as hv
hv.extension('bokeh')

import numpy as np

def interpolate_guess(domain_from,domain_to,rank,guess):
    '''Helper for upscaling the intial guesses'''
    guess = guess.reshape((domain_from.length,rank,rank))
    new_guess = np.zeros((domain_to.length,rank,rank))
    for i in range(rank):
        for j in range(rank):
            new_guess[:,i,j] = np.interp(domain_to.r,domain_from.r,guess[:,i,j])
    return new_guess.reshape((-1,))

In [ ]:
%opts Curve Scatter [width=500,height=400] Layout [shared_axes=False] Scatter (size=10,alpha=0.5)
%opts Curve Scatter [fontsize={'xlabel':14,'xlabel':14,'ylabel':14,'ticks':12}]
%opts Overlay [legend_position='bottom_left']
%opts Layout [shared_axes=False]


colors = {}
colors['linear',10] = 'blue'
colors['linear',60] = 'red'
colors['comb',10] = 'green'
colors['comb',60] = 'magenta'

ls = {}
ls['linear',10] = 'solid'
ls['linear',60] = 'dashed'
ls['comb',10] = 'dotted'
ls['comb',60] = 'dashdot'


markers = {}
markers['linear',10] = 'o'
markers['linear',60] = '^'
markers['comb',10] = 'd'
markers['comb',60] = 's'


## Demo

First we load the reference data

In [ ]:
PMF_compare = []
for arch in ['linear','comb']:
    for Nmatrix in [10,60]:
        fname = './data/GraftedComposite-Gr-{}-Nmatrix{}-PP.dat'.format(arch,Nmatrix)
        x,y = np.loadtxt(fname).T
        y = -np.log(y)
        PMF_compare.append([arch,Nmatrix,x,y])

Solving the PRISM problem...

In [ ]:
import typyPRISM
from typyPRISM.calculate.prism.pair_correlation import pair_correlation
from typyPRISM.calculate.prism.pmf import pmf
import numpy as np

D = 5.0
d = 1.0
phi = 0.001
eta = 0.35
vd = 4.0/3.0 * np.pi * (d/2.0)**(3)
vD = 4.0/3.0 * np.pi * (D/2.0)**(3)


sys = typyPRISM.System(['particle','graft','matrix'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.1,length=1024)

sys.diameter['particle'] = D
sys.diameter['graft']    = d
sys.diameter['matrix']   = d

sys.omega['particle','matrix']  = typyPRISM.omega.InterMolecular()
sys.omega['graft','matrix']     = typyPRISM.omega.InterMolecular()
sys.omega['particle','particle'] = typyPRISM.omega.SingleSite()

sys.closure[sys.types,sys.types]   = typyPRISM.closure.PercusYevick()
sys.closure['particle','particle'] = typyPRISM.closure.HyperNettedChain()

sys.potential[sys.types,sys.types]   = typyPRISM.potential.HardSphere(sigma=d)
sys.potential['particle',sys.types]  = typyPRISM.potential.HardSphere(sigma=(d + D)/2.0)
sys.potential['particle','particle'] = typyPRISM.potential.HardSphere(sigma=D)

guess = np.zeros((sys.domain.length,sys.rank,sys.rank))
PMF_results = []
for Nmatrix in [60,10]:
    sys.omega['matrix','matrix']   = typyPRISM.omega.FromFile('./data/GraftedComposite-Omega-matrix-N{}.dat'.format(Nmatrix))
    sys.omega['graft','graft']     = typyPRISM.omega.FromFile('./data/GraftedComposite-Omega-linear-GG.dat')
    sys.omega['graft','particle']  = typyPRISM.omega.FromFile('./data/GraftedComposite-Omega-linear-GP.dat')
    
    numGraftBeads = 500
    vPGP = vD + numGraftBeads*vd
    sys.density['matrix']   = (1-phi)*eta/vd
    sys.density['particle'] = phi*eta/vPGP
    sys.density['graft']    = numGraftBeads * phi*eta/vPGP
    print('--> rho=',sys.density['graft'],sys.density['particle'],sys.density['matrix'])
    PRISM = sys.createPRISM()
    PRISM.solve(guess)
    guess = np.copy(PRISM.x)
    PMF_results.append(['linear',Nmatrix,sys.domain.r,pmf(PRISM)['particle','particle']])
    
    numGraftBeads = 1250
    vPGP = vD + numGraftBeads*vd
    sys.density['matrix']  = (1-phi)*eta/vd
    sys.density['particle'] = phi*eta/vPGP
    sys.density['graft']    = numGraftBeads * phi*eta/vPGP
    print('--> rho=',sys.density['graft'],sys.density['particle'],sys.density['matrix'])
    sys.omega['matrix','matrix']   = typyPRISM.omega.FromFile('./data/GraftedComposite-Omega-matrix-N{}.dat'.format(Nmatrix))
    sys.omega['graft','graft']     = typyPRISM.omega.FromFile('./data/GraftedComposite-Omega-comb-GG.dat')
    sys.omega['graft','particle']  = typyPRISM.omega.FromFile('./data/GraftedComposite-Omega-comb-GP.dat')
    PRISM = sys.createPRISM()
    result = PRISM.solve(guess)
    PMF_results.append(['comb',Nmatrix,sys.domain.r,pmf(PRISM)['particle','particle']])


Plot the results!

In [ ]:
%%opts Overlay [legend_position='top_right']
from math import sqrt

extents = (10,-0.2,40,0.2)
gr_plots = []
for arch,Nmatrix,x,y in PMF_results:
    key = (arch,Nmatrix)
    label = 'alpha={} (typyPRISM)'.format(key)
    style = {'line_dash':ls[key],'color':colors[key]}
    c1 = hv.Curve((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
step = 10
for arch,Nmatrix,x,y in PMF_compare:
    x = x[::step]
    y = y[::step]
    key = (arch,Nmatrix)
    label = 'alpha={} (Ref 1)'.format(key)
    style = {'marker':markers[key],'color':colors[key]}
    c1 = hv.Scatter((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
    
hv.Overlay(gr_plots).redim.label(x='r',y='g(r)')